In [1]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import *
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import urllib
import os
import time

### Q1) Write a python program which searches all the product under a particular product verticalfrom www.amazon.in. The product verticals to be searched will be taken as input from user.For e.g. If user input is ‘guitar’. Then search for guitars.

In [52]:
driver = webdriver.Chrome(r"S:\chromedriver_win32\chromedriver.exe")

In [3]:
driver.get('http://www.amazon.in/')
search = driver.find_element_by_xpath("//input[@id='twotabsearchtextbox']")
time.sleep(2)
search.clear()
product = input("Please Enter the product you want to search: ")
search.send_keys(product)
btn = driver.find_element_by_xpath("//input[@id='nav-search-submit-button']")
btn.click()

Please Enter the product you want to search: Guitar


### Q2) In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“. 

In [4]:
upper = int(input("Enter the number of pages you want to scrape: "))
upper = upper + 1

Enter the number of pages you want to scrape: 3


In [5]:
Link = []

In [6]:
def links():
    for i in range(0,upper):
        try:
            link = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none a-spacing-top-small']/h2/a")
            for j in link:
                Link.append(j.get_attribute('href'))
            nxt = driver.find_element_by_xpath("//li[@class='a-last']")
            nxt.click()
            time.sleep(5)
        except NoSuchElementException as e:
            print("There are no more pages for this product")

In [7]:
links()

In [8]:
Brand = []
Name = []
Price = []
Return = []
Expected=[]
Avaliable = []
More = []
Ratings = []
conv = []
Rate = []
Star = []

In [9]:
def data():
    for i in Link:
        driver.get(i)
        time.sleep(2)
        br = driver.find_elements_by_xpath("//div[@class='a-expander-content a-expander-section-content a-section-expander-inner']/table/tbody/tr/td")
        for j in br:
            Brand.append(j.text)
        name = driver.find_elements_by_xpath("//span[@id='productTitle']")
        for k in name:
            Name.append(k.text)
        price = driver.find_elements_by_xpath("//span[@id='priceblock_ourprice']")
        for l in price:
            Price.append(l.text)
        ret = driver.find_elements_by_xpath("//a[@class='a-size-small a-link-normal a-text-normal']")
        for m in ret:
            Return.append(m.text)
        expect = driver.find_elements_by_xpath("//div[@id='ddmDeliveryMessage']/b")
        for n in expect:
            Expected.append(n.text)
        avaliable = driver.find_elements_by_xpath("//div[@id='availability']/span")
        for o in avaliable:
            Avaliable.append(o.text)
        more = driver.find_elements_by_xpath("//div[@id='feature-bullets']/ul")
        for p in more:
            More.append(p.text)
        ratings = driver.find_elements_by_xpath("//td[@class='a-size-base']")
        for q in ratings:
            Ratings.append(q.text)
        for r in Ratings:
            conv.append(r.replace("\n", " "))
        for s in conv:
            Rate.append(s.split()[0])
            Star.append(s.split()[2])
        
driver.get('https://www.amazon.in/')

In [10]:
data()

In [11]:
data = {
    'Brand':Brand,
    'Name of Product as per website':Name,
    'Ratings':Rate,
    'Stars':Star,
    'Price':Price,
    'Return/Exchange Time':Return,
    'Expected Delivery': Expected,
    'Avaliability':Avaliable,
    'Other Details':More,
    'Product Link':Link
}

In [12]:
Amazon = pd.DataFrame.from_dict(data, orient='index')

In [13]:
Amazon = Amazon.transpose()

In [14]:
Amazon = Amazon.dropna(thresh=3)

In [15]:
Amazon = Amazon.fillna('-')

In [16]:
Amazon.to_csv("Amazon.csv", encoding='utf-8', header='Amazon Data')

### Q3) Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [17]:
def search():
    driver.get("https://images.google.com/")
    product = input("Enter the name of the thing you want to search: ")
    search = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
    search.send_keys(product)
    btn = driver.find_element_by_xpath("//button[@class='Tg7LZd']")
    btn.click()

In [18]:
def download(j):
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
        time.sleep(2)
        new_height = driver.execute_script('return document.body.scrollHeight')
        try:
            driver.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input').click()
            time.sleep(2)
        except:
            pass
        if new_height == last_height:
            break
        last_height = new_height
        
    
    for i in range(1,102):
        try:
            driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div['+str(i)+']/a[1]/div[1]/img').screenshot('S:\Data Science Projects\Flip Robo\Web_Scraping_3 ('+str(i)+str(j)+').png')
        except:
            pass

In [19]:
upper = int(input("How many image topic you want to scrape: "))
upper = upper + 1
for i in range(1,upper):
    j = input("Enter the name of the file: ")
    search()
    download(j)

How many image topic you want to scrape: 3
Enter the name of the file: Fruits
Enter the name of the thing you want to search: Fruits
Enter the name of the file: Cars
Enter the name of the thing you want to search: Cars
Enter the name of the file: Machine Learning
Enter the name of the thing you want to search: Machine Learning


### Q4) Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) onwww.flipkart.com and scrape following details for all the search results displayed on 1st page.Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”,“Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “DisplayResolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”.Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [53]:
driver.get("http://www.flipkart.com/")

In [54]:
close = driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']")
close.click()

In [55]:
product = input("Enter the Product name you want to scrape: ")
search = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
search.clear()
search.send_keys(product)
btn = driver.find_element_by_xpath("//button[@class='L0Z3Pu']")
btn.click()

Enter the Product name you want to scrape: Samsung


In [69]:
Link = []

In [70]:
def links():
    link = driver.find_elements_by_xpath("//div[@class='_2kHMtA']/a")
    for i in link:
        Link.append(i.get_attribute('href'))

In [71]:
links()

In [72]:
MobileName = []
Price = []
Box = []
ModelNo = []
Modelname = []
Colour = []
Simtype = []
TouchScreen = []
OTG = []
QuickCharging = []
DisplaySize = []
Resolution = []
ResolutionType = []
GPU = []
OS = []
Processor = []
Core = []
PrimaryClock = []
SecondaryClock = []
InternalStorage = []
RAM = []
PrimaryCamera = []
SecondaryCamera = []
BatteryCapacity = []
BatteryType = []

In [73]:
def data():
    for i in Link:
        driver.get(i)
        time.sleep(2)
        try:
            btn = driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']")
            btn.click()
            time.sleep(3)
            price = driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
            Price.append(price.text)
            mobilename = driver.find_element_by_xpath("//span[@class='B_NuCI']")
            MobileName.append(mobilename.text)
            box = driver.find_element_by_xpath("//table[@class='_14cfVK']/tbody/tr[1]/td[2]")
            Box.append(box.text)
            model = driver.find_element_by_xpath("//table[@class='_14cfVK']/tbody/tr[2]/td[2]")
            ModelNo.append(model.text)
            mname = driver.find_element_by_xpath("//table[@class='_14cfVK']/tbody/tr[3]/td[2]")
            Modelname.append(mname.text)
            colour = driver.find_element_by_xpath("//table[@class='_14cfVK']/tbody/tr[4]/td[2]")
            Colour.append(colour.text)
            sim = driver.find_element_by_xpath("//table[@class='_14cfVK']/tbody/tr[6]/td[2]")
            Simtype.append(sim.text)
            touch = driver.find_element_by_xpath("//table[@class='_14cfVK']/tbody/tr[8]/td[2]")
            TouchScreen.append(touch.text)
            otg = driver.find_element_by_xpath("//table[@class='_14cfVK']/tbody/tr[9]/td[2]")
            OTG.append(otg.text)
            quick = driver.find_element_by_xpath("//table[@class='_14cfVK']/tbody/tr[10]/td[2]")
            QuickCharging.append(quick.text)
            display = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[2]/table/tbody/tr[1]/td[2]")
            DisplaySize.append(display.text)
            resolution = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[2]/table/tbody/tr[2]/td[2]")
            Resolution.append(resolution.text)
            resolutiontype = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[2]/table/tbody/tr[3]/td[2]")
            ResolutionType.append(resolutiontype.text)
            gpu = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[2]/table/tbody/tr[4]/td[2]")
            GPU.append(gpu.text)
            os = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[3]/table/tbody/tr[1]/td[2]")
            OS.append(os.text)
            processor = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[3]/table/tbody/tr[2]/td[2]")
            Processor.append(processor.text)
            core = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[3]/table/tbody/tr[3]/td[2]")
            Core.append(core.text)
            primary = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[3]/table/tbody/tr[4]/td[2]")
            PrimaryClock.append(primary.text)
            secondary = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[3]/table/tbody/tr[5]/td[2]")
            SecondaryClock.append(secondary.text)
            internal = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[4]/table/tbody/tr[1]/td[2]")
            InternalStorage.append(internal.text)
            ram = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[4]/table/tbody/tr[2]/td[2]")
            RAM.append(ram.text)
            primarycamera = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[5]/table/tbody/tr[2]/td[2]")
            PrimaryCamera.append(primarycamera.text)
            secondarycamera = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[5]/table/tbody/tr[6]/td[2]")
            SecondaryCamera.append(secondarycamera.text)
            battery = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[10]/table/tbody/tr[1]/td[2]")
            BatteryCapacity.append(battery.text)
            batterytype = driver.find_element_by_xpath("//div[@class='_1UhVsV']/div[10]/table/tbody/tr[2]/td[2]")
            BatteryType.append(batterytype.text)
        except:
            pass

In [74]:
data()

In [75]:
info = {
    'Mobile Name':MobileName,
    'Price':Price,
    'Inside the box':Box,
    'Model Number': ModelNo,
    'Model Name': Modelname,
    'Colour': Colour,
    'Sim Type': Simtype,
    'Touch Screen': TouchScreen,
    'OTG': OTG,
    'Quick Charging': QuickCharging,
    'Display Size': DisplaySize,
    'Resolution': Resolution,
    'Resolution Type': ResolutionType,
    'GPU': GPU,
    'OS':OS,
    'Processor': Processor,
    'Core': Core,
    'Primary Clock Speed' : PrimaryClock,
    'Secondary Clock Speed': SecondaryClock,
    'Internal Storage': InternalStorage,
    'RAM': RAM,
    'Primary Camera': PrimaryCamera,
    'Ssecondary Camera': SecondaryCamera,
    'Battery Capacity': BatteryCapacity,
    'Battery Type': BatteryType
}

In [76]:
Mobile = pd.DataFrame.from_dict(info, orient='index').transpose()

In [77]:
Mobile.fillna("-", inplace = True)

In [79]:
Mobile.to_csv("Flipkart.csv", encoding='utf-8', header='Mobile')

### Q5) Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [80]:
driver.get("https://www.google.com/maps")

In [81]:
def city():
    cityname = input("Enter the city for scraping the coordinates: ")
    search = driver.find_element_by_xpath("//div[@id='gs_lc50']/input[1]")
    search.clear()
    search.send_keys(cityname)
    btn = driver.find_element_by_xpath("//button[@id='searchbox-searchbutton']")
    btn.click()

In [82]:
Latitude = []
Longitude = []
current = []

In [83]:
def geo():
    current = driver.current_url
    half = current.split("@")[1]
    Latitude.append(half.split(",")[0])
    Longitude.append(half.split(",")[1])
    print("Latitude of searched city is: ",Latitude)
    print("Longitude of Searched city is: ", Longitude)

In [84]:
city()

Enter the city for scraping the coordinates: Delhi


In [85]:
geo()

Latitude of searched city is:  ['18.8154265']
Longitude of Searched city is:  ['76.7751434']


###  Q6) Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [86]:
driver.get("https://trak.in/")

In [87]:
fd = driver.find_element_by_xpath("//li[@id='menu-item-51510']")
fd.click()

In [88]:
Date  =[]
date = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[2]")
for i in date:
    Date.append(i.text)

In [89]:
StartupName = []
start = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[3]")
for i in start:
    StartupName.append(i.text)

In [90]:
Industry = []
industry = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[4]")
for i in industry:
    Industry.append(i.text)

In [91]:
Sub = []
sub = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[5]")
for i in sub:
    Sub.append(i.text)

In [92]:
City = []
city = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[6]")
for i in city:
    City.append(i.text)

In [93]:
Investors = []
investor = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[7]")
for i in investor:
    Investors.append(i.text)

In [94]:
InvType = []
inv = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[8]")
for i in inv:
    InvType.append(i.text)

In [95]:
Amount = []
amount = driver.find_elements_by_xpath("//table[@id='tablepress-48']/tbody/tr/td[9]")
for i in amount:
    Amount.append(i.text)

In [96]:
data = {
    'Date':Date,
    'Start-Up Name': StartupName,
    'Industry': Industry,
    'Sub Industry' : Sub,
    'City' : City,
    'Investors' : Investors,
    'Investment Type': InvType,
    'Amount' : Amount
}

In [97]:
July20 = pd.DataFrame.from_dict(data)

In [98]:
Date  =[]
date = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[2]")
for i in date:
    Date.append(i.text)

In [99]:
StartupName = []
start = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[3]")
for i in start:
    StartupName.append(i.text)

In [100]:
Industry = []
industry = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[4]")
for i in industry:
    Industry.append(i.text)

In [101]:
Sub = []
sub = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[5]")
for i in sub:
    Sub.append(i.text)

In [102]:
City = []
city = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[6]")
for i in city:
    City.append(i.text)

In [103]:
Investors = []
investor = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[7]")
for i in investor:
    Investors.append(i.text)

In [104]:
InvType = []
inv = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[8]")
for i in inv:
    InvType.append(i.text)

In [105]:
Amount = []
amount = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[9]")
for i in amount:
    Amount.append(i.text)

In [106]:
data = {
    'Date':Date,
    'Start-Up Name': StartupName,
    'Industry': Industry,
    'Sub Industry' : Sub,
    'City' : City,
    'Investors' : Investors,
    'Investment Type': InvType,
    'Amount' : Amount
}

In [107]:
August20 = pd.DataFrame.from_dict(data)

In [108]:
Date  =[]
date = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[2]")
for i in date:
    Date.append(i.text)

In [109]:
StartupName = []
start = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[3]")
for i in start:
    StartupName.append(i.text)

In [110]:
Industry = []
industry = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[4]")
for i in industry:
    Industry.append(i.text)

In [111]:
Sub = []
sub = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[5]")
for i in sub:
    Sub.append(i.text)

In [112]:
City = []
city = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[6]")
for i in city:
    City.append(i.text)

In [113]:
Investors = []
investor = driver.find_elements_by_xpath("//table[@id='tablepress-49']/tbody/tr/td[7]")
for i in investor:
    Investors.append(i.text)

In [114]:
InvType = []
inv = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[8]")
for i in inv:
    InvType.append(i.text)

In [115]:
Amount = []
amount = driver.find_elements_by_xpath("//table[@id='tablepress-50']/tbody/tr/td[9]")
for i in amount:
    Amount.append(i.text)

In [116]:
data = {
    'Date':Date,
    'Start-Up Name': StartupName,
    'Industry': Industry,
    'Sub Industry' : Sub,
    'City' : City,
    'Investors' : Investors,
    'Investment Type': InvType,
    'Amount' : Amount
}

In [117]:
September20 = pd.DataFrame.from_dict(data)

In [118]:
SecondQuater = July20.append(August20)

In [119]:
SecondQuater = SecondQuater.append(September20)

In [120]:
SecondQuater.to_csv("Funding Deals of 2nd Quater July 2020 - September 2020.csv", encoding='utf-8', header='Funding Deals of 2nd Quater July 2020 - September 2020')

### Q3) Write a program to scrap all the available details of top 10 gaming laptops from digit.in.  

In [121]:
driver.get("https://www.digit.in/")

In [122]:
top = driver.find_element_by_xpath("//div[@class='menu']/ul/li[4]")
top.click()
laptop = driver.find_element_by_xpath("//div[@class='listing_container']/ul/li[26]")
laptop.click()

In [123]:
Laptop = []
laptop = driver.find_elements_by_xpath("//div[@class='right-container']/div/a")
for i in laptop:
    Laptop.append(i.text)

In [124]:
Operating = []
os = driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]")
for i in os:
    Operating.append(i.text)

In [125]:
Display = []
display = driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]")
for i in display:
    Display.append(i.text)

In [126]:
Processor = []
processor = driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]")
for i in processor:
    Processor.append(i.text)

In [127]:
Memory = []
memory = driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[4]")
for i in memory:
    Memory.append(i.text)

In [128]:
Weight = []
weight = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[7]")
for i in weight:
    Weight.append(i.text)

In [129]:
Dimension = []
dimension = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[8]/td[3]")
for i in dimension:
    Dimension.append(i.text)

In [130]:
Graphics = []
graphics = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[9]/td[3]")
for i in graphics:
    Graphics.append(i.text)

In [131]:
Description = []
description = driver.find_elements_by_xpath("//div[@class='Section-center']/p")
for i in description:
    Description.append(i.text)

In [132]:
Links = []
links = driver.find_elements_by_xpath("//div[@class='TopNumbeHeading sticky-footer']/a")
for i in links:
    Links.append(i.get_attribute('href'))

In [133]:
Gaming = {
    'Name of Laptop' : Laptop,
    'Operating' : Operating,
    'Display' : Display,
    'Processor' : Processor,
    'Memory' : Memory, 
    'Weight' : Weight,
    'Dimension' : Dimension,
    'Graphics' : Graphics,
    'Description' : Description,
    'Product Link' : Links
}

In [134]:
TopGaming  = pd.DataFrame.from_dict(Gaming, orient='index').transpose()

In [135]:
TopGaming.fillna("-", inplace=True)

In [136]:
TopGaming.to_csv("Topgaming.csv", encoding='utf-8', header='Top Gaming Laptop')